In [8]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import geemap
import ee

In [4]:
ee.Authenticate()
ee.Initialize(project='twoearthengineproject')

In [5]:
# Load AOI shapefile
physio_koshi = gpd.read_file(r"G:\learn\python_works\Work_projects\Koshi_chetan\chetan_cas\works\data\gis\koshi_basin_nepal\koshi_nepal_basin.shp")
whole_koshi_basin = physio_koshi.dissolve()
ee_physio_koshi = ee.FeatureCollection(geemap.geopandas_to_ee(whole_koshi_basin))

# Load Landsat collections
l4 = ee.ImageCollection("LANDSAT/LT04/C02/T1_L2")
l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
l9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")

# Cloud masking functions
def mask_tm(image):
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(1 << 3).eq(0).And(qa.bitwiseAnd(1 << 4).eq(0))
    return image.updateMask(mask).multiply(0.0000275).add(-0.2)

def mask_oli(image):
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(1 << 3).eq(0).And(qa.bitwiseAnd(1 << 4).eq(0))
    return image.updateMask(mask).multiply(0.0000275).add(-0.2)

# Composite function
def get_composite(year):
    start = ee.Date.fromYMD(year, 1, 1)
    end = ee.Date.fromYMD(year, 12, 31)

    if year < 2013:
        col = (l4.filterBounds(ee_physio_koshi).filterDate(start, end).map(mask_tm)
               .merge(l5.filterBounds(ee_physio_koshi).filterDate(start, end).map(mask_tm))
               .merge(l7.filterBounds(ee_physio_koshi).filterDate(start, end).map(mask_tm)))
    else:
        col = (l8.filterBounds(ee_physio_koshi).filterDate(start, end).map(mask_oli)
               .merge(l9.filterBounds(ee_physio_koshi).filterDate(start, end).map(mask_oli)))

    # Mosaic fills in all available pixels
    return col.mosaic().clip(ee_physio_koshi)

# Years to process
years = [1990, 2000, 2010, 2020, 2024]

# Create map
Map = geemap.Map()
Map.centerObject(ee_physio_koshi, 7)

for y in years:
    img = get_composite(y)
    
    # Add layer for visualization
    if y < 2013:
        vis_params = {'bands': ['SR_B5', 'SR_B4', 'SR_B3'], 'min': 0, 'max': 0.3}
    else:
        vis_params = {'bands': ['SR_B6', 'SR_B5', 'SR_B4'], 'min': 0, 'max': 0.3}
    Map.addLayer(img, vis_params, f"Landsat_{y}")


Map

Map(center=[27.459741378291714, 86.82980172772386], controls=(WidgetControl(options=['position', 'transparent_…

In [12]:
ee.Authenticate()

True

In [ ]:

ee.Initialize(project='giskoshi')

AttributeError: module 'ee.data' has no attribute 'authenticateViaPopup'

In [10]:
# Load AOI shapefile
physio_koshi = gpd.read_file(r"G:\learn\python_works\Work_projects\Koshi_chetan\chetan_cas\works\data\gis\koshi_basin_nepal\koshi_nepal_basin.shp")
whole_koshi_basin = physio_koshi.dissolve()
ee_physio_koshi = ee.FeatureCollection(geemap.geopandas_to_ee(whole_koshi_basin))

# Load Landsat collections
l4 = ee.ImageCollection("LANDSAT/LT04/C02/T1_L2")
l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
l9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")

# Cloud masking functions
def mask_tm(image):
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(1 << 3).eq(0).And(qa.bitwiseAnd(1 << 4).eq(0))
    return image.updateMask(mask).multiply(0.0000275).add(-0.2)

def mask_oli(image):
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(1 << 3).eq(0).And(qa.bitwiseAnd(1 << 4).eq(0))
    return image.updateMask(mask).multiply(0.0000275).add(-0.2)

# Composite function
def get_composite(year):
    start = ee.Date.fromYMD(year, 1, 1)
    end = ee.Date.fromYMD(year, 12, 31)

    if year < 2013:
        col = (l4.filterBounds(ee_physio_koshi).filterDate(start, end).map(mask_tm)
               .merge(l5.filterBounds(ee_physio_koshi).filterDate(start, end).map(mask_tm))
               .merge(l7.filterBounds(ee_physio_koshi).filterDate(start, end).map(mask_tm)))
    else:
        col = (l8.filterBounds(ee_physio_koshi).filterDate(start, end).map(mask_oli)
               .merge(l9.filterBounds(ee_physio_koshi).filterDate(start, end).map(mask_oli)))

    return col.mosaic().clip(ee_physio_koshi)

# Years to process
years = [1990]

# Export each composite to Google Drive
for y in years:
    img = get_composite(y)
    
    # Define export parameters
    export_params = {
        'scale': 30,  # 30m resolution (adjust if needed)
        'region': ee_physio_koshi.geometry().bounds(),  # Export the entire AOI
        'maxPixels': 1e13,  # Increase if needed
        'fileFormat': 'GeoTIFF',
        'folder': 'Landsat_Koshi_Basin',  # Folder in Google Drive
        'fileNamePrefix': f'Landsat_{y}_AllBands'  # Output filename
    }
    
    # Start the export task
    task = ee.batch.Export.image.toDrive(img, **export_params)
    task.start()
    print(f"Exporting Landsat {y} to Google Drive...")

print("All tasks submitted. Check Google Drive for downloads.")

EEException: Caller does not have required permission to use project giskoshi. Grant the caller the roles/serviceusage.serviceUsageConsumer role, or a custom role with the serviceusage.services.use permission, by visiting https://console.developers.google.com/iam-admin/iam/project?project=giskoshi and then retry. Propagation of the new permission may take a few minutes.

In [ ]:
# Export each composite to Google Drive
for y in years:
    img = get_composite(y)
    
    # Define export parameters
    export_params = {
        'scale': 30,  # 30m resolution (adjust if needed)
        'region': ee_physio_koshi.geometry().bounds(),  # Export the entire AOI
        'maxPixels': 1e13,  # Increase if needed
        'fileFormat': 'GeoTIFF',
        'folder': 'Landsat_Koshi_Basin',  # Folder in Google Drive
        'fileNamePrefix': f'Landsat_{y}_AllBands'  # Output filename
    }
    
    # Start the export task
    task = ee.batch.Export.image.toDrive(img, **export_params)
    task.start()
    print(f"Exporting Landsat {y} to Google Drive...")

print("All tasks submitted. Check Google Drive for downloads.")

Generating URL ...
An error occurred while downloading.
Total request size (11652251904 bytes) must be less than or equal to 50331648 bytes.
Downloaded Landsat 1990 to Landsat_Koshi_Downloads\Landsat_1990_AllBands.tif
Generating URL ...
An error occurred while downloading.
Total request size (11652251904 bytes) must be less than or equal to 50331648 bytes.
Downloaded Landsat 2000 to Landsat_Koshi_Downloads\Landsat_2000_AllBands.tif
Generating URL ...
An error occurred while downloading.
Total request size (11652251904 bytes) must be less than or equal to 50331648 bytes.
Downloaded Landsat 2010 to Landsat_Koshi_Downloads\Landsat_2010_AllBands.tif
Generating URL ...
An error occurred while downloading.
Total request size (11652251904 bytes) must be less than or equal to 50331648 bytes.
Downloaded Landsat 2020 to Landsat_Koshi_Downloads\Landsat_2020_AllBands.tif
Generating URL ...
An error occurred while downloading.
Total request size (11652251904 bytes) must be less than or equal to 503

In [ ]:
# Load AOI shapefile
physio_koshi = gpd.read_file(r"G:\learn\python_works\Work_projects\Koshi_chetan\chetan_cas\works\data\gis\koshi_basin_nepal\koshi_nepal_basin.shp")
whole_koshi_basin = physio_koshi.dissolve()
ee_physio_koshi = ee.FeatureCollection(geemap.geopandas_to_ee(whole_koshi_basin))

# Load Landsat collections
l4 = ee.ImageCollection("LANDSAT/LT04/C02/T1_L2")
l5 = ee.ImageCollection("LANDSAT/LT05/C02/T1_L2")
l7 = ee.ImageCollection("LANDSAT/LE07/C02/T1_L2")
l8 = ee.ImageCollection("LANDSAT/LC08/C02/T1_L2")
l9 = ee.ImageCollection("LANDSAT/LC09/C02/T1_L2")

# Cloud masking functions
def mask_tm(image):
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(1 << 3).eq(0).And(qa.bitwiseAnd(1 << 4).eq(0))
    return image.updateMask(mask).multiply(0.0000275).add(-0.2)

def mask_oli(image):
    qa = image.select('QA_PIXEL')
    mask = qa.bitwiseAnd(1 << 3).eq(0).And(qa.bitwiseAnd(1 << 4).eq(0))
    return image.updateMask(mask).multiply(0.0000275).add(-0.2)

# Composite function
def get_composite(year):
    start = ee.Date.fromYMD(year, 1, 1)
    end = ee.Date.fromYMD(year, 12, 31)

    if year < 2013:
        col = (l4.filterBounds(ee_physio_koshi).filterDate(start, end).map(mask_tm)
               .merge(l5.filterBounds(ee_physio_koshi).filterDate(start, end).map(mask_tm))
               .merge(l7.filterBounds(ee_physio_koshi).filterDate(start, end).map(mask_tm)))
    else:
        col = (l8.filterBounds(ee_physio_koshi).filterDate(start, end).map(mask_oli)
               .merge(l9.filterBounds(ee_physio_koshi).filterDate(start, end).map(mask_oli)))

    return col.mosaic().clip(ee_physio_koshi)

# Years to process
years = [1990, 2000, 2010, 2020, 2024]

# Export each composite to Google Drive
for y in years:
    img = get_composite(y)
    
    # Define export parameters
    export_params = {
        'scale': 30,  # 30m resolution (adjust if needed)
        'region': ee_physio_koshi.geometry().bounds(),  # Export the entire AOI
        'maxPixels': 1e13,  # Increase if needed
        'fileFormat': 'GeoTIFF',
        'folder': 'Landsat_Koshi_Basin',  # Folder in Google Drive
        'fileNamePrefix': f'Landsat_{y}_AllBands'  # Output filename
    }
    
    # Start the export task
    task = ee.batch.Export.image.toDrive(img, **export_params)
    task.start()
    print(f"Exporting Landsat {y} to Google Drive...")

print("All tasks submitted. Check Google Drive for downloads.")